In [1]:
using Plots
using Printf

各数値を定義

In [2]:
g = 9.8
m1 = 1.0
m2 = 1.0
l1 = 1.0
l2 = 1.0

1.0

力学的エネルギーを計算する関数の定義

In [3]:
function  calc_mechanical_energy(y)
    θ1, θ2, ω1, ω2 = y
    return -m1 * g * l1 * cos(θ1) - m2 * g * ((l1 * cos(θ1)) + l2 * cos(θ2)) + 0.5 * m1 * l1^2 * ω1^2 + 0.5 * m2 * (l1^2 * ω1^2 + l2^2 * ω2^2 + 2 * l1 * l2 * ω1 * ω2 * cos(θ1 - θ2))
end

calc_mechanical_energy (generic function with 1 method)

微分方程式を定義

In [4]:
function pendulum_dynamics(y, t)
    θ1, θ2, ω1, ω2 = y
    C = cos(θ1 - θ2)
    S = sin(θ1 - θ2)
    M = m1 + m2
    θ1_dot = ω1
    θ2_dot = ω2
    ω1_dot = (g * (C * sin(θ2) - M * sin(θ1) / m2) - (l1 * C * ω1^2 + l2 * ω2^2) * S) / (l1 * (M / m2 - C^2))
    ω2_dot = (g * (M / m2) * (C * sin(θ1) - sin(θ2)) + ((M / m2) * l1 * ω1^2 + l2 * ω2^2 * C) *S) / (l2 * (M / m2 - C^2))
    return [θ1_dot, θ2_dot, ω1_dot, ω2_dot]
end

pendulum_dynamics (generic function with 1 method)

オイラー法

In [5]:
function euler(f, y0, tspan, dt)
    t0, tf = tspan
    t = t0:dt:tf
    y = zeros(length(t), length(y0))
    y[1, :] = y0

    for i = 1:length(t)-1
        y[i+1, :] = y[i, :] + f(y[i, :], t[i]) * dt
    end
    
    return t, y
    
end

euler (generic function with 1 method)

ルンゲクッタ法

In [6]:
function runge_kutta(f, y0, tspan, dt)
    t0, t_end = tspan
    t = t0:dt:t_end
    y = zeros(length(t), length(y0))
    y[1, :] = y0

    for i in 1:length(t) - 1
        k1 = f(y[i, :], t[i])
        k2 = f(y[i, :] + 0.5 * dt * k1, t[i] + 0.5 * dt)
        k3 = f(y[i, :] + 0.5 * dt * k2, t[i] + 0.5 * dt)
        k4 = f(y[i, :] + dt * k3, t[i] + dt)
        y[i + 1, :] = y[i, :] + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    end

    return t, y

end

runge_kutta (generic function with 1 method)

初期条件

In [16]:
y0 = [π/4, 0.0, π/2, 0.0] # [θ1, ω1, θ2, ω2]
tspan = (0.0, 1.0)
dt = 0.01

0.01

シミュレーション実行

In [17]:
t_euler, y_euler = euler(pendulum_dynamics, y0, tspan, dt)
t_runge_kutta, y_runge_kutta = runge_kutta(pendulum_dynamics, y0, tspan, dt)

(0.0:0.01:1.0, [0.7853981633974483 0.0 1.5707963267948966 0.0; 0.8006040686858401 0.00043715384933415685 1.4704837909105002 0.08685605997944415; … ; -0.31357650985203356 0.18289832733707095 0.8967604031056032 -4.420451311418229; -0.3039077591567242 0.13796904383878694 1.0366698750854648 -4.565098149126998])

誤差を計算

In [18]:
initial_energy = calc_mechanical_energy(y0)

energy_euler = [calc_mechanical_energy(y) for y in eachrow(y_euler)]
error_euler = [abs(energy - initial_energy) for energy in energy_euler]

energy_runge_kutta = [calc_mechanical_energy(y) for y in eachrow(y_runge_kutta)]
error_runge_kutta = [abs(energy - initial_energy) for energy in energy_runge_kutta]



101-element Vector{Float64}:
 0.0
 1.28836674662125e-9
 1.1036185298962664e-9
 6.02895511292445e-11
 1.8437980031649204e-9
 3.989619301592029e-9
 6.317623757468027e-9
 8.7057117070799e-9
 1.1075577788233204e-8
 1.3382361885305727e-8
 ⋮
 2.3295399742551126e-7
 2.831456242802233e-7
 3.5622488070430336e-7
 4.5886342903145305e-7
 5.972670642506728e-7
 7.74315928708802e-7
 9.84110556601081e-7
 1.2031391882771914e-6
 1.3788264041636467e-6

In [14]:
println(maximum(error_euler))

6.136438592956651


In [20]:
plot(
    t_euler, error_euler, label="Euler", xlabel="Time [s]", ylabel="Error in Energy[J]", 
    title="Error in Energy vs Time", lw=2
)
plot!(
    t_runge_kutta, error_runge_kutta, label="Runge-Kutta", lw=2
)
savefig("error_vs_time.png")



"c:\\numerical_calculation\\computational_physics\\first_report\\error_vs_time.png"